# Packages

In [ ]:
import os
import logging
import pandas as pd
import geopandas as gpd
import networkx as nx
import osmnx as ox
import matplotlib.pyplot as plt
import numpy as np
import requests
import json
import fiona

from itertools import combinations

from shapely.geometry import Point
from shapely.ops import unary_union
from shapely.ops import nearest_points

import folium
from folium import GeoJson, LayerControl
from branca.colormap import linear

# Functions

In [ ]:
def tag_flooded_roads(edges, nodes, flood_zones, layer):
    output_path = flood_cut_roads[layer]
    graphml_path = flood_safe_roads[layer]

    if os.path.exists(output_path) and layer in fiona.listlayers(output_path):
        print(f"Loading {layer} from {output_path}")
        edges = gpd.read_file(output_path, layer=layer)
    else:
        print(f"Tagging and saving {layer} to {output_path}")

        bounds = edges.total_bounds
        flood_subset = flood_zones.cx[bounds[0]:bounds[2], bounds[1]:bounds[3]]
        flood_geoms = flood_subset.geometry

        edges = edges.copy()
        edges["in_flood_zone"] = edges.geometry.apply(lambda geom: flood_geoms.intersects(geom).any())

        edges.to_file(output_path, layer=layer, driver="GPKG")

    if os.path.exists(graphml_path):
        print("Pruned graph already exists")
    else:    
        safe_edges = edges[~edges["in_flood_zone"]].copy()
        
        # Rebuild graph from safe edges
        print("Rebuilding pruned graph...")
        G_safe = ox.graph_from_gdfs(nodes, safe_edges)
        ox.save_graphml(G_safe, filepath=graphml_path)
        print(f"Saved pruned graph to {graphml_path}")

    return edges, G_safe

In [ ]:
def load_or_clip_flood_zone(return_crs, layer, source_path, clip_geom):
    output_path = flood_zones[layer]
    
    if os.path.exists(output_path):
        print(f"Loading {layer} from {output_path}")
        clipped = gpd.read_file(output_path, layer=layer).to_crs(return_crs)
    else:
        # Clip and save the original
        print(f"Clipping and saving {layer} from {output_path}" )
        flood = gpd.read_file(source_path).to_crs(return_crs)
        clipped = gpd.clip(flood, clip_geom)
        clipped.to_file(output_path, layer=layer, driver="GPKG")

    return clipped

In [ ]:
def make_geojson_safe(gdf):
    gdf = gdf.copy()
    dt_cols = gdf.select_dtypes(include=['datetime64[ns]', 'datetime64[ns, UTC]']).columns
    gdf[dt_cols] = gdf[dt_cols].astype(str)
    for col in gdf.columns:
        if col != "geometry" and not pd.api.types.is_scalar(gdf[col].iloc[0]):
            gdf.drop(columns=[col], inplace=True)
    return gdf

In [ ]:
def add_flood_zone_layer(gdf, name, color, m):
    if gdf.crs.to_epsg() != 4326:
        gdf = gdf.to_crs(epsg=4326)

    gdf_serializable = make_geojson_safe(gdf)

    style_function = lambda x: {
        'fillColor': color,
        'color': color,
        'weight': 1,
        'fillOpacity': 0.4
    }

    geojson = folium.GeoJson(
        data=gdf_serializable,
        name=f"Flood {name}",
        style_function=style_function,
        show=False
    )
    geojson.add_to(m)

In [ ]:
def add_flooded_roads_layer(edges, name, color, m):
    flooded = edges[edges["in_flood_zone"] == True]
    flooded = flooded.to_crs(epsg=4326)
    flooded = make_geojson_safe(flooded)

    style_function = lambda x: {
        'color': color,
        'weight': 2,
        'opacity': 0.8
    }

    geojson = folium.GeoJson(
        flooded,
        name=f"Flooded Roads {name}",
        style_function=style_function,
        show=False
    )
    geojson.add_to(m)

In [ ]:
def add_roads_layer(edges, name, color, m):
    roads = edges.copy()
    roads = roads.to_crs(epsg=4326)
    roads = make_geojson_safe(roads)

    style_function = lambda x: {
        'color': color,
        'weight': 2,
        'opacity': 0.8
    }

    geojson = folium.GeoJson(
        roads,
        name=f"{name}",
        style_function=style_function,
        show=False
    )
    geojson.add_to(m)

In [ ]:
def parse_depth_range(val):
    if pd.isna(val):
        return None

    val = val.strip()

    if val.startswith('Below'):
        return float(val[5:].strip()) / 2

    if val.startswith('>'):
        return float(val[1:].strip())  # You may want to cap it

    if '-' in val:
        parts = val.split('-')
        try:
            low = float(parts[0].strip())
            high = float(parts[1].strip())
            return (low + high) / 2
        except:
            return None

    try:
        return float(val)
    except:
        return None

In [ ]:
def shortest_path_length(G, source, target, node_to_muni):
    try:
        path = nx.shortest_path(G, source=source, target=target, weight='length')
        total_length = nx.shortest_path_length(G, source=source, target=target, weight='length')
        key = f"{node_to_muni[source]}__{node_to_muni[target]}"

    except (nx.NetworkXNoPath, nx.NodeNotFound):
        print(f"No path between {source} and {target} ({node_to_muni[source]} - {node_to_muni[target]})")
        path = ""
        total_length = 0
        key = f"{node_to_muni[source]}__{node_to_muni[target]}"
    return key, path, total_length
    

In [ ]:
def calculate_shortest_paths(G, output_dir, path_filename):
    os.makedirs(output_dir, exist_ok=True)
    municipal_nodes = [node for node, data in G.nodes(data=True) if data.get("municipality")]
    municipal_nodes = sorted(municipal_nodes)
    node_to_muni = {node: G.nodes[node]['municipality'] for node in municipal_nodes}
    path_path = os.path.join(output_dir, path_filename)
    
    if os.path.exists(path_path):
        logging.info(f"Shortest paths already calculated at {path_path}")
        return
    
    logging.info(f"Calculating shortest paths between all municipalities for graph...")
    
    paths_dict = {}
    for source, target in combinations(municipal_nodes, 2):
        key, path, length = shortest_path_length(G, source, target, node_to_muni)
        paths_dict[key] = {
            "nodes": path,
            "total_length": length
        }
    
    with open(path_path, "w") as f:
        json.dump(paths_dict, f, indent=2)
    
    logging.info(f"Saved shortest paths to {path_path}")

In [ ]:
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")

# Street data

In [ ]:
output_dir = "processed_files"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
ox.settings.use_cache = True
ox.settings.log_console = False

In [ ]:
polygon_path = os.path.join(output_dir, "study_area.geojson")
graph_path = os.path.join(output_dir, "tagged_graph.graphml")

valencia_municipalities = [
    "Alaquàs", "Albal", "Albalat de la Ribera", "Alberic", "Alborache", "Alcàsser", "l'Alcúdia",
    "Aldaia", "Alfafar", "Alfarb", "Algemesí", "Alginet", "Almussafes", "Alzira",
    "Benetússer", "Benicull de Xúquer", "Benifaió", "Beniparrell", "Benimodo", "Bétera",
    "Bugarra", "Buñol", "Calles", "Camporrobles", "Carcaixent", "Carlet", "Catadau", "Catarroja",
    "Caudete de las Fuentes", "Chera", "Cheste", "Chiva", "Chulilla", "Corbera", "Cullera",
    "Dos Aguas", "Favara", "Fortaleny", "Fuenterrobles", "Gestalgar", "Godelleta", "Guadassuar",
    "l'Ènova", "Llaurí", "Llombai", "Llíria", "Llocnou de la Corona", "Loriguilla", "Macastre",
    "Manises", "Manuel", "Massanassa", "Millares", "Mislata", "Montroi", "Montserrat", "Paiporta",
    "Paterna", "Pedralba", "Picanya", "Picassent", "Polinyà de Xúquer", "La Pobla Llarga",
    "Quart de Poblet", "Rafelguaraf", "Real", "Requena", "Riba-roja de Túria", "Riola", "Sedaví",
    "Senyera", "Siete Aguas", "Silla", "Sinarcas", "Sollana", "Sot de Chera", "Sueca",
    "Tavernes de la Valldigna", "Torrent", "Tous", "Turís", "Utiel", "València", "Vilamarxant", "Xirivella",
    "Yátova"
]

In [ ]:
if os.path.exists(polygon_path):
    logging.info("Loading saved study area polygon...")
    study_area = gpd.read_file(polygon_path)
else:
    logging.info("Downloading polygons for municipalities...")
    polygons = []

    for municipality in valencia_municipalities:
        try:
            place_name = f"{municipality}, Valencia, Spain"
            gdf = ox.geocode_to_gdf(place_name)

            if gdf.crs != "EPSG:4326":
                gdf = gdf.to_crs("EPSG:4326")

            polygons.append(gdf)
        except Exception as e:
            logging.warning(f"Error retrieving {municipality}: {e}", exc_info=True)

    # Add Mira from Cuenca (just in case)
    try:
        gdf = ox.geocode_to_gdf("Mira, Cuenca, Spain")
        if gdf.crs != "EPSG:4326":
            gdf = gdf.to_crs("EPSG:4326")
        polygons.append(gdf)
    except Exception as e:
        logging.warning(f"Error retrieving Mira: {e}", exc_info=True)

    study_area = gpd.GeoDataFrame(pd.concat(polygons, ignore_index=True), crs="EPSG:4326")
    study_area = study_area[study_area.geometry.notnull()]
    study_area.to_file(polygon_path, driver="GeoJSON")

In [ ]:
study_area = study_area[study_area.geometry.type.isin(["Polygon", "MultiPolygon"])]
polygon = unary_union(study_area.geometry)
logging.info("Polygon geometry union complete.")

In [ ]:
if os.path.exists(graph_path):
    logging.info("Loading saved road network graph...")
    G = ox.load_graphml(graph_path)
else:
    logging.info("Downloading road network...")
    G = ox.graph_from_polygon(polygon, network_type="drive", simplify=True)
    ox.save_graphml(G, filepath=graph_path)
    logging.info("Graph saved.")

nodes, edges = ox.graph_to_gdfs(G)
logging.info("Converted graph to GeoDataFrames.")

In [ ]:
from scipy.spatial import cKDTree

if 'municipality' not in nodes.columns:
    logging.info("Adding 'municipality' field to nodes...")

    nodes['municipality'] = ""

    node_coords = np.array([(geom.y, geom.x) for geom in nodes.geometry])
    kdtree = cKDTree(node_coords)

    for _, row in study_area.iterrows():
        name = row.get("name") or row.get("display_name") or "unknown"
        geom = row.geometry

        if not geom or not geom.is_valid or name == "Favara":
            continue

        try:
            muni_graph = ox.graph_from_polygon(geom, network_type='drive', simplify=True)
            center_node = ox.distance.nearest_nodes(muni_graph, X=geom.centroid.x, Y=geom.centroid.y)
            center_point = Point((muni_graph.nodes[center_node]['x'], muni_graph.nodes[center_node]['y']))
            _, idx = kdtree.query([center_point.y, center_point.x], k=1)
            nearest_node_idx = nodes.index[idx]
            nodes.at[nearest_node_idx, 'municipality'] = name

        except Exception as e:
            logging.warning(f"Could not process {name}: {e}")
            continue

    for node_id, row in nodes.iterrows():
        G.nodes[node_id]['municipality'] = row['municipality']

    graph_path = os.path.join(output_dir, "tagged_graph.graphml")
    ox.save_graphml(G, graph_path)
    logging.info(f"Updated graph saved to: {graph_path}")

else:
    logging.info("'municipality' field already exists in nodes.")

In [ ]:
calculate_shortest_paths(G, output_dir, path_filename="shortest_paths_length.json")

In [ ]:
rail=False
if rail:
    graph_path_rail = os.path.join(output_dir, "study_area_rail.graphml")
    if os.path.exists(graph_path_rail):
        print("Loading saved rail network graph...")
        G_rail = ox.load_graphml(graph_path_rail)
    else:
        print("Downloading rail network graph...")
        rail_filter = '["railway"~"rail|light_rail|subway|tram"]'
        G_rail = ox.graph_from_polygon(polygon, custom_filter=rail_filter, network_type="all")
        ox.save_graphml(G_rail, filepath=graph_path_rail)

    nodes_rail, edges_rail = ox.graph_to_gdfs(G_rail)
    del G_rail

# Layers

In [ ]:
flood_zones = {
    "10 yr": f"{output_dir}/flood_risk_zone_10.gpkg",
    "100 yr": f"{output_dir}/flood_risk_zone_100.gpkg",
    "500 yr": f"{output_dir}/flood_risk_zone_500.gpkg",
    "DANA_02_11_2024": f"{output_dir}/DANA_zone_02_11_2024.gpkg",
    "DANA_03_11_2024": f"{output_dir}/DANA_zone_03_11_2024.gpkg",
    "DANA_06_11_2024": f"{output_dir}/DANA_zone_06_11_2024.gpkg",
    "DANA_08_11_2024": f"{output_dir}/DANA_zone_08_11_2024.gpkg",
    "DANA depth": f"{output_dir}/DANA_depths.gpkg"
}

flood_cut_roads = {
    "10 yr": f"{output_dir}/flood_risk_cut_roads_10.gpkg",
    "100 yr": f"{output_dir}/flood_risk_cut_roads_100.gpkg",
    "500 yr": f"{output_dir}/flood_risk_cut_roads_500.gpkg",
    "DANA_02_11_2024": f"{output_dir}/DANA_cut_roads_02_11_2024.gpkg",
    "DANA_03_11_2024": f"{output_dir}/DANA_cut_roads_03_11_2024.gpkg",
    "DANA_06_11_2024": f"{output_dir}/DANA_cut_roads_06_11_2024.gpkg",
    "DANA_08_11_2024": f"{output_dir}/DANA_cut_roads_08_11_2024.gpkg"
}

flood_safe_roads = {
    "10 yr": f"{output_dir}/flood_risk_10_safe_roads.graphml",
    "100 yr": f"{output_dir}/flood_risk__100_safe_roads.graphml",
    "500 yr": f"{output_dir}/flood_risk__500_safe_roads.graphml",
    "DANA_02_11_2024": f"{output_dir}/DANA_safe_roads_02_11_2024.graphml",
    "DANA_03_11_2024": f"{output_dir}/DANA_safe_roads_03_11_2024.graphml",
    "DANA_06_11_2024": f"{output_dir}/DANA_safe_roads_06_11_2024.graphml",
    "DANA_08_11_2024": f"{output_dir}/DANA_safe_roads_08_11_2024.graphml"
}

# Floodable zones

In [ ]:
flood_risk_zone_10  = load_or_clip_flood_zone(edges.crs, "10 yr", "source_files/laminaspb-q10/Q10_2Ciclo_PB_20241121.shp", polygon)
flood_risk_zone_100 = load_or_clip_flood_zone(edges.crs, "100 yr", "source_files/laminaspb-q100/Q100_2Ciclo_PB_20241121_ETRS89.shp", polygon)
flood_risk_zone_500 = load_or_clip_flood_zone(edges.crs, "500 yr", "source_files/laminaspb-q500/Q500_2Ciclo_PB_20241121_ETRS89.shp", polygon)

In [ ]:
edges_flood_10, G_flood_10 = tag_flooded_roads(edges, nodes, flood_risk_zone_10, "10 yr")
edges_flood_100, G_flood_100 = tag_flooded_roads(edges, nodes, flood_risk_zone_100, "100 yr")
edges_flood_500, G_flood_500 = tag_flooded_roads(edges, nodes, flood_risk_zone_500, "500 yr")

# Flooded Area

In [ ]:
output_dir = "processed_files"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
flood_zone_DANA_02_11_2024  = load_or_clip_flood_zone(edges.crs, "DANA_02_11_2024", "source_files/EMSR773_AOI01_DEL_PRODUCT_v1/EMSR773_AOI01_DEL_PRODUCT_observedEventA_v1.shp", polygon)
flood_zone_DANA_03_11_2024  = load_or_clip_flood_zone(edges.crs, "DANA_03_11_2024", "source_files/EMSR773_AOI01_DEL_MONIT01_v1/EMSR773_AOI01_DEL_MONIT01_observedEventA_v1.shp", polygon)
flood_zone_DANA_06_11_2024  = load_or_clip_flood_zone(edges.crs, "DANA_06_11_2024", "source_files/EMSR773_AOI01_DEL_MONIT02_v1/EMSR773_AOI01_DEL_MONIT02_observedEventA_v1.shp", polygon)
flood_zone_DANA_08_11_2024  = load_or_clip_flood_zone(edges.crs, "DANA_08_11_2024", "source_files/EMSR773_AOI01_DEL_MONIT04_v1/EMSR773_AOI01_DEL_MONIT04_observedEventA_v1.shp", polygon)

In [ ]:
edges_DANA_02_11_2024, G_DANA_02_11_2024 = tag_flooded_roads(edges, nodes, flood_zone_DANA_02_11_2024, "DANA_02_11_2024")
edges_DANA_03_11_2024, G_DANA_03_11_2024 = tag_flooded_roads(edges, nodes, flood_zone_DANA_03_11_2024, "DANA_03_11_2024")
edges_DANA_06_11_2024, G_DANA_06_11_2024 = tag_flooded_roads(edges, nodes, flood_zone_DANA_06_11_2024, "DANA_06_11_2024")
edges_DANA_08_11_2024, G_DANA_08_11_2024 = tag_flooded_roads(edges, nodes, flood_zone_DANA_08_11_2024, "DANA_08_11_2024")

In [ ]:
graphs = {
    "DANA_02_11_2024": G_DANA_02_11_2024,
    "DANA_03_11_2024": G_DANA_03_11_2024,
    "DANA_06_11_2024": G_DANA_06_11_2024,
    "DANA_08_11_2024": G_DANA_08_11_2024
}


for name, graph in graphs.items():
    calculate_shortest_paths(graph, output_dir, "shortest_path_length_"+name+".json")

In [ ]:
layer="DANA depth"
output_path = flood_zones[layer]

if os.path.exists(output_path) and layer in fiona.listlayers(output_path):
    print(f"Loading {layer} from {output_path}")
    DANA_flood_depth=gpd.read_file(output_path, layer=layer)
else:
    print(f"Saving {layer} to {output_path}")
    DANA_flood_depth = gpd.read_file("source_files/EMSR773_AOI01_DEL_PRODUCT_v1/EMSR773_AOI01_DEL_PRODUCT_floodDepthA_v1.shp")
    DANA_flood_depth["depth_val"] = DANA_flood_depth["value"].apply(parse_depth_range)
    DANA_flood_depth.to_file(output_path, layer=layer, driver="GPKG")
    print(f"Saved processed {layer} in {output_path}")

# Interactive Visualizations

In [ ]:
# Set initial position
projected = study_area.to_crs(epsg=25830)
centroid_projected = projected.geometry.centroid.iloc[0]
centroid_latlon = gpd.GeoSeries([centroid_projected], crs=25830).to_crs(epsg=4326).geometry.iloc[0]
map_center = [centroid_latlon.y, centroid_latlon.x]
bounds_wgs84 = study_area.to_crs(epsg=4326).total_bounds
map_bounds = [[bounds_wgs84[1], bounds_wgs84[0]], [bounds_wgs84[3], bounds_wgs84[2]]]

## Areas at Risk and DANA Area (with roads)

In [ ]:
flood_colors = {
    "10 yr": "#56B4E9",   # Sky Blue
    "100 yr": "#009E73",  # Bluish Green
    "500 yr": "#E69F00",  # Orange
    "DANA_02_11_2024": "#CC79A7"  # Reddish Purple
}  

In [ ]:
m_1 = folium.Map(location=map_center, zoom_start=10, tiles="CartoDB positron", max_bounds=True)
m_1.fit_bounds(map_bounds)

In [ ]:
# Add flood zones
add_flood_zone_layer(flood_risk_zone_10, "10 yr", flood_colors["10 yr"], m_1)
add_flood_zone_layer(flood_risk_zone_100, "100 yr", flood_colors["100 yr"], m_1)
add_flood_zone_layer(flood_risk_zone_500, "500 yr", flood_colors["500 yr"], m_1)
add_flood_zone_layer(flood_zone_DANA_02_11_2024, "DANA_02_11_2024", flood_colors["DANA_02_11_2024"], m_1)
    
# Add flooded roads (optional)
add_flooded_roads_layer(edges_flood_10, "10 yr", flood_colors["10 yr"], m_1)
add_flooded_roads_layer(edges_flood_100, "100 yr", flood_colors["100 yr"], m_1)
add_flooded_roads_layer(edges_flood_500, "500 yr", flood_colors["500 yr"], m_1)
add_flooded_roads_layer(edges_DANA_02_11_2024, "DANA_02_11_2024", flood_colors["DANA_02_11_2024"], m_1)
add_roads_layer(edges, "All Roads", "#000000", m_1)

In [ ]:
folium.LayerControl(collapsed=False).add_to(m_1)
m_1.save("processed_files/m_1.html")

## DANA flood depth

In [ ]:
m_2 = folium.Map(location=map_center, zoom_start=10, tiles="CartoDB positron", max_bounds=True)
m_2.fit_bounds(map_bounds)

In [ ]:
min_depth = DANA_flood_depth["depth_val"].min()
max_depth = DANA_flood_depth["depth_val"].max()
depth_colormap = linear.YlGnBu_09.scale(min_depth, max_depth)
depth_colormap.caption = 'Flood Depth (m)'

folium.GeoJson(
    DANA_flood_depth,
    name="DANA flood depth",
    style_function=lambda feature: {
        'fillColor': depth_colormap(feature['properties']['depth_val']),
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.7
    },
    tooltip=folium.GeoJsonTooltip(fields=["depth_val"], aliases=["Depth (m):"])
).add_to(m_2)

depth_colormap.add_to(m_2)

In [ ]:
folium.LayerControl(collapsed=False).add_to(m_2)
m_2.save("processed_files/m_3.html")

## DANA Flooded Area Evolution

In [ ]:
flood_colors = {
    "DANA_02_11_2024": "#56B4E9",   # Sky Blue
    "DANA_03_11_2024": "#009E73",  # Bluish Green
    "DANA_06_11_2024": "#E69F00",  # Orange
    "DANA_08_11_2024": "#CC79A7"  # Reddish Purple
}  

In [ ]:
m_3 = folium.Map(location=map_center, zoom_start=10, tiles="CartoDB positron", max_bounds=True)
m_3.fit_bounds(map_bounds)

In [ ]:
# Add flood zones
add_flood_zone_layer(flood_zone_DANA_02_11_2024, "DANA_02_11_2024", flood_colors["DANA_02_11_2024"], m_3)
add_flood_zone_layer(flood_zone_DANA_03_11_2024, "DANA_03_11_2024", flood_colors["DANA_03_11_2024"], m_3)
add_flood_zone_layer(flood_zone_DANA_06_11_2024, "DANA_06_11_2024", flood_colors["DANA_06_11_2024"], m_3)
add_flood_zone_layer(flood_zone_DANA_08_11_2024, "DANA_08_11_2024", flood_colors["DANA_08_11_2024"], m_3)
    
# Add flooded roads (optional)
add_flooded_roads_layer(edges_DANA_02_11_2024, "DANA_02_11_2024", flood_colors["DANA_02_11_2024"], m_3)
add_flooded_roads_layer(edges_DANA_03_11_2024, "DANA_03_11_2024", flood_colors["DANA_03_11_2024"], m_3)
add_flooded_roads_layer(edges_DANA_06_11_2024, "DANA_06_11_2024", flood_colors["DANA_06_11_2024"], m_3)
add_flooded_roads_layer(edges_DANA_08_11_2024, "DANA_08_11_2024", flood_colors["DANA_08_11_2024"], m_3)
add_roads_layer(edges, "All Roads", "#000000", m_3)

In [ ]:
folium.LayerControl(collapsed=False).add_to(m_3)
m_3.save("processed_files/m_3.html")

# Old Code